<a href="https://colab.research.google.com/github/kr1k-boop/mini-project-ST-/blob/main/knnskl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df= pd.read_csv('/content/drive/MyDrive/finale (2).csv')

In [4]:
df.head()

,Unnamed: 0,acousticness,name,danceability,tempo,loudness,energy,id,instrumentalness,key,liveness,popularity,speechiness,valence,year,userid,playcount
0,0,0.995,Singende Bataillone 1. Teil Carl Woitschach,0.708,0.002377,-0.002373,0.1950,6KbQ3uYMLKb5jDxLF7wYDD,0.563,10.0,0.1510,0.0,0.0506,0.7790,1928.0,109679.0,5.0
1,1,0.994,"Fantasiestücke, Op. 111: Più tosto lento Rober...",0.379,0.001685,-0.005434,0.0135,6KuQTIu1KoTTkLXKrwlLPV,0.901,8.0,0.0763,0.0,0.0462,0.0767,1928.0,13441.0,7.0
2,2,0.994,"Fantasiestücke, Op. 111: Più tosto lento Rober...",0.379,0.001685,-0.005434,0.0135,2YPbmbRIo7I1r9FAMqNjbZ,0.901,8.0,0.0763,0.0,0.0462,0.0767,1928.0,25637.0,3.0
3,3,0.604,Chapter 1.18 - Zamek kaniowski Seweryn Goszczy...,0.749,0.002150,-0.003805,0.2200,6L63VW0PibdM1HDSBoqnoM,0.000,5.0,0.1190,0.0,0.9290,0.8800,1928.0,63548.0,7.0
4,4,0.995,Bebamos Juntos - Instrumental (Remasterizado) ...,0.781,0.002167,-0.002814,0.1300,6M94FkXd15sOAOQYRnWPN8,0.887,1.0,0.1110,0.0,0.0926,0.7200,1928.0,10501.0,8.0


In [5]:
df.dtypes

Unnamed: 0            int64
acousticness        float64
name                 object
danceability        float64
tempo               float64
loudness            float64
energy              float64
id                   object
instrumentalness    float64
key                 float64
liveness            float64
popularity          float64
speechiness         float64
valence             float64
year                float64
userid              float64
playcount           float64
dtype: object

In [6]:
df.describe()

,Unnamed: 0,acousticness,danceability,tempo,loudness,energy,instrumentalness,key,liveness,popularity,speechiness,valence,year,userid,playcount
count,1.135383e+06,1.135383e+06,1.135383e+06,1.135383e+06,1.135383e+06,1.135383e+06,1.135383e+06,1.135383e+06,1.135383e+06,1.135383e+06,1.135383e+06,1.135383e+06,1.135383e+06,1.135383e+06,1.135383e+06
mean,3.324539e+05,3.656228e-01,5.564450e-01,2.389023e-03,-1.838428e-03,5.700818e-01,1.160923e-01,5.227812e+00,1.969572e-01,4.124020e+01,8.368191e-02,5.287928e-01,1.986701e+03,5.489920e+04,4.170311e+00
std,1.994362e+05,3.642513e-01,1.710036e-01,5.968195e-04,1.013417e-03,2.607855e-01,2.661965e-01,3.534076e+00,1.651970e-01,2.223515e+01,1.251396e-01,2.513564e-01,2.383429e+01,3.172225e+04,4.790470e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.145770e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.921000e+03,0.000000e+00,1.000000e+00
25%,1.603630e+05,3.050000e-02,4.410000e-01,1.926604e-03,-2.374608e-03,3.650000e-01,0.000000e+00,2.000000e+00,9.510000e-02,2.700000e+01,3.380000e-02,3.250000e-01,1.970000e+03,2.741600e+04,1.000000e+00
50%,3.108820e+05,2.090000e-01,5.660000e-01,2.366232e-03,-1.589374e-03,6.100000e-01,6.480000e-05,5.000000e+00,1.300000e-01,4.500000e+01,4.370000e-02,5.360000e-01,1.995000e+03,5.483200e+04,3.000000e+00
75%,5.167425e+05,7.260000e-01,6.760000e-01,2.766356e-03,-1.090582e-03,7.840000e-01,1.700000e-02,8.000000e+00,2.550000e-01,5.800000e+01,7.120000e-02,7.360000e-01,2.006000e+03,8.234600e+04,6.000000e+00
max,6.607190e+05,9.960000e-01,9.880000e-01,4.897285e-03,7.361570e-04,1.000000e+00,1.000000e+00,1.100000e+01,1.000000e+00,1.000000e+02,9.690000e-01,1.000000e+00,2.020000e+03,1.099990e+05,5.820000e+02


In [7]:
df.drop(['Unnamed: 0'],axis=1, inplace=True)

In [8]:
df.head()

,acousticness,name,danceability,tempo,loudness,energy,id,instrumentalness,key,liveness,popularity,speechiness,valence,year,userid,playcount
0,0.995,Singende Bataillone 1. Teil Carl Woitschach,0.708,0.002377,-0.002373,0.1950,6KbQ3uYMLKb5jDxLF7wYDD,0.563,10.0,0.1510,0.0,0.0506,0.7790,1928.0,109679.0,5.0
1,0.994,"Fantasiestücke, Op. 111: Più tosto lento Rober...",0.379,0.001685,-0.005434,0.0135,6KuQTIu1KoTTkLXKrwlLPV,0.901,8.0,0.0763,0.0,0.0462,0.0767,1928.0,13441.0,7.0
2,0.994,"Fantasiestücke, Op. 111: Più tosto lento Rober...",0.379,0.001685,-0.005434,0.0135,2YPbmbRIo7I1r9FAMqNjbZ,0.901,8.0,0.0763,0.0,0.0462,0.0767,1928.0,25637.0,3.0
3,0.604,Chapter 1.18 - Zamek kaniowski Seweryn Goszczy...,0.749,0.002150,-0.003805,0.2200,6L63VW0PibdM1HDSBoqnoM,0.000,5.0,0.1190,0.0,0.9290,0.8800,1928.0,63548.0,7.0
4,0.995,Bebamos Juntos - Instrumental (Remasterizado) ...,0.781,0.002167,-0.002814,0.1300,6M94FkXd15sOAOQYRnWPN8,0.887,1.0,0.1110,0.0,0.0926,0.7200,1928.0,10501.0,8.0


In [9]:
df.shape

(1135383, 16)

In [10]:
le = LabelEncoder()
n_id= le.fit_transform(df['id'])

In [11]:
df.insert(15, 'song_id', n_id)

In [12]:
df.drop('id', axis=1, inplace=True)

In [13]:
df.head()

,acousticness,name,danceability,tempo,loudness,energy,instrumentalness,key,liveness,popularity,speechiness,valence,year,userid,song_id,playcount
0,0.995,Singende Bataillone 1. Teil Carl Woitschach,0.708,0.002377,-0.002373,0.1950,0.563,10.0,0.1510,0.0,0.0506,0.7790,1928.0,109679.0,140959,5.0
1,0.994,"Fantasiestücke, Op. 111: Più tosto lento Rober...",0.379,0.001685,-0.005434,0.0135,0.901,8.0,0.0763,0.0,0.0462,0.0767,1928.0,13441.0,141059,7.0
2,0.994,"Fantasiestücke, Op. 111: Più tosto lento Rober...",0.379,0.001685,-0.005434,0.0135,0.901,8.0,0.0763,0.0,0.0462,0.0767,1928.0,25637.0,60998,3.0
3,0.604,Chapter 1.18 - Zamek kaniowski Seweryn Goszczy...,0.749,0.002150,-0.003805,0.2200,0.000,5.0,0.1190,0.0,0.9290,0.8800,1928.0,63548.0,141124,7.0
4,0.995,Bebamos Juntos - Instrumental (Remasterizado) ...,0.781,0.002167,-0.002814,0.1300,0.887,1.0,0.1110,0.0,0.0926,0.7200,1928.0,10501.0,141446,8.0


In [14]:
X = df.copy()

In [15]:
X.head()

,acousticness,name,danceability,tempo,loudness,energy,instrumentalness,key,liveness,popularity,speechiness,valence,year,userid,song_id,playcount
0,0.995,Singende Bataillone 1. Teil Carl Woitschach,0.708,0.002377,-0.002373,0.1950,0.563,10.0,0.1510,0.0,0.0506,0.7790,1928.0,109679.0,140959,5.0
1,0.994,"Fantasiestücke, Op. 111: Più tosto lento Rober...",0.379,0.001685,-0.005434,0.0135,0.901,8.0,0.0763,0.0,0.0462,0.0767,1928.0,13441.0,141059,7.0
2,0.994,"Fantasiestücke, Op. 111: Più tosto lento Rober...",0.379,0.001685,-0.005434,0.0135,0.901,8.0,0.0763,0.0,0.0462,0.0767,1928.0,25637.0,60998,3.0
3,0.604,Chapter 1.18 - Zamek kaniowski Seweryn Goszczy...,0.749,0.002150,-0.003805,0.2200,0.000,5.0,0.1190,0.0,0.9290,0.8800,1928.0,63548.0,141124,7.0
4,0.995,Bebamos Juntos - Instrumental (Remasterizado) ...,0.781,0.002167,-0.002814,0.1300,0.887,1.0,0.1110,0.0,0.0926,0.7200,1928.0,10501.0,141446,8.0


In [16]:
X.drop(['acousticness', 'danceability', 'tempo', 'loudness', 'energy', 'instrumentalness', 'key', 'liveness', 'popularity','speechiness', 'valence', 'year'], axis = 1, inplace=True)

In [17]:
X.head()

,name,userid,song_id,playcount
0,Singende Bataillone 1. Teil Carl Woitschach,109679.0,140959,5.0
1,"Fantasiestücke, Op. 111: Più tosto lento Rober...",13441.0,141059,7.0
2,"Fantasiestücke, Op. 111: Più tosto lento Rober...",25637.0,60998,3.0
3,Chapter 1.18 - Zamek kaniowski Seweryn Goszczy...,63548.0,141124,7.0
4,Bebamos Juntos - Instrumental (Remasterizado) ...,10501.0,141446,8.0


In [18]:
combine_song_play = X.dropna(axis = 0, subset = ['name'])
song_playcount = (combine_song_play.
     groupby(by = ['name'])['playcount'].
     count().
     reset_index().
     rename(columns = {'playcount': 'totalPlayCount'})
     [['name', 'totalPlayCount']]
    )

In [19]:
song_playcount.head()

,name,totalPlayCount
0,Cello Song Nick Drake,4
1,!Que Vida! - Mono Version Love,4
2,"""40"" - Live U2",4
3,"""40"" - Remastered 2008 U2",8
4,"""40"" U2",4


In [20]:
songs_with_totalPlayCount = combine_song_play.merge(song_playcount, left_on='name', right_on='name', how='left')

In [21]:
songs_with_totalPlayCount.head()

,name,userid,song_id,playcount,totalPlayCount
0,Singende Bataillone 1. Teil Carl Woitschach,109679.0,140959,5.0,4
1,"Fantasiestücke, Op. 111: Più tosto lento Rober...",13441.0,141059,7.0,8
2,"Fantasiestücke, Op. 111: Più tosto lento Rober...",25637.0,60998,3.0,8
3,Chapter 1.18 - Zamek kaniowski Seweryn Goszczy...,63548.0,141124,7.0,4
4,Bebamos Juntos - Instrumental (Remasterizado) ...,10501.0,141446,8.0,4


In [22]:
print(songs_with_totalPlayCount['totalPlayCount'].describe())

count    1.135383e+06
mean     4.373580e+02
std      1.208086e+03
min      1.000000e+00
25%      4.000000e+00
50%      8.000000e+00
75%      1.670000e+02
max      7.614000e+03
Name: totalPlayCount, dtype: float64


In [ ]:
song_features = songs_with_totalPlayCount.pivot_table(index='name',columns='userid', values='playcount').fillna(0)